In [1]:
1+1

2

In [2]:
import mozo2 

import importlib
importlib.reload(mozo2)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from mozo import compare_and_combine_2, Diff, merge_with_features, num_to_range_categories
import pandas as pd
import numpy as np
np.random.seed(0)

import xgboost as xgb

from scipy import stats
import math 
import datetime 

from sklearn.metrics import r2_score, mean_absolute_error
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from functools import partial
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from sklearn.svm import SVC

from sklearn.metrics import r2_score, f1_score, mean_absolute_error, mean_squared_error

from scikitplot.estimators import plot_learning_curve

import time
from datetime import datetime


In [3]:
## modelowanie

In [4]:
# import warnings
# warnings.filterwarnings("ignore")

total = pd.read_csv('~/pml7/konkurs/output/newest_total_20201101_07_20.h5.csv')
total.info(verbose=True)

train_30 = pd.read_hdf('output/train_30.h5', 'data') 
test_car_ids = train_30['car_id'].values

train_70 = total[~total['car_id'].isin(test_car_ids) & ~total['price_value'].isna()]
train_30 = total[total['car_id'].isin(test_car_ids)]

print(len(total.index))
print(len(total[~total['price_value'].isna()].index))

print(train_70['price_value'].isna().sum())
print(train_30['price_value'].isna().sum())

train_100 = total[~total['price_value'].isna()]
print(train_100['price_value'].isna().sum())

global_min = total[~total['price_value'].isna()].price_value.min()

test = total[total['price_value'].isna()]
train = total[~total['price_value'].isna()]


/home/conda/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,86,161) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160616 entries, 0 to 160615
Data columns (total 218 columns):
 #   Column                                                  Dtype  
---  ------                                                  -----  
 0   breadcrumb                                              object 
 1   created_at                                              object 
 2   price_currency                                          object 
 3   price_details                                           object 
 4   price_value                                             float64
 5   seller_address                                          object 
 6   seller_name                                             object 
 7   seller_type                                             object 
 8   feature_czujniki_parkowania_przednie                    bool   
 9   feature_poduszka_powietrzna_chroniąca_kolana            bool   
 10  feature_kurtyny_powietrzne                             

160616
106494
0
0
0


In [10]:
start_vars = np.sort([
# 'created_at_unix', 'encoded_param_napęd_2', 'encoded_param_rodzaj_paliwa_2', 'encoded_param_skrzynia_biegów_2', 'feature_abs', 'feature_alufelgi', 'feature_asr__kontrola_trakcji_', 'feature_asystent_parkowania', 'feature_bluetooth', 'feature_cd', 'feature_czujniki_parkowania_przednie', 'feature_elektryczne_szyby_tylne', 'feature_elektrycznie_ustawiane_lusterka', 'feature_esp__stabilizacja_toru_jazdy_', 'feature_gniazdo_sd', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_dwustrefowa', 'feature_klimatyzacja_manualna', 'feature_radio_fabryczne', 'feature_tempomat', 'feature_światła_do_jazdy_dziennej', 'feature_światła_led', 'feature_światła_xenonowe', 'latitude', 'longitude', 'model_XGBRegressor_5856_36229_', 'new_param_kod_silnika_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_model_pojazdu_cat', 'new_param_napęd_cat', 'new_param_oferta_od_cat', 'new_param_vin_cat', 'new_param_wersja_cat', 'new_seller_name_cat', 'param_bezwypadkowy', 'param_ile_lat_temu_produkcja', 'param_kierownica_po_prawej__anglik_', 'param_leasing', 'param_liczba_miejsc_2', 'param_moc', 'param_pierwsza_rejestracja_date_na', 'param_pojemność_skokowa', 'param_przebieg', 'param_rok_produkcji', 'param_serwisowany_w_aso', 'param_tuning', 'param_uszkodzony', 'param_wartość_wykupu_2', 'param_zarejestrowany_w_polsce', 'price_currency_pln'
# 'created_at_unix', 'encoded_param_rodzaj_paliwa_2', 'encoded_param_typ_2', 'feature_abs', 'feature_asr__kontrola_trakcji_', 'feature_bluetooth', 'feature_cd', 'feature_czujniki_parkowania_przednie', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_gniazdo_sd', 'feature_hud__wyświetlacz_przezierny_', 'feature_isofix', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_manualna', 'feature_ogrzewanie_postojowe', 'feature_podgrzewane_tylne_siedzenia', 'feature_relingi_dachowe', 'feature_tapicerka_skórzana', 'feature_tapicerka_welurowa', 'feature_tempomat', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'feature_światła_xenonowe', 'latitude_na_mean', 'longitude', 'longitude_na_mean', 'model_8790', 'new_param_kolor_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_oferta_od_cat', 'new_param_opłata_początkowa_cat', 'new_param_rodzaj_paliwa_cat', 'new_param_wersja_cat', 'new_seller_name_cat', 'new_vin_flex_fuel_cat', 'new_vin_model_cat', 'new_vin_security_cat', 'param_bezwypadkowy', 'param_homologacja_ciężarowa', 'param_ile_lat_temu_produkcja', 'param_leasing', 'param_liczba_miejsc_2', 'param_liczba_pozostałych_rat_2', 'param_matowy', 'param_przebieg', 'param_rok_produkcji', 'param_tuning', 'param_uszkodzony', 'param_wartość_wykupu_2'
# 'model_8790', 'feature_bluetooth', 'created_at_unix', 'encoded_param_rodzaj_paliwa_2', 'encoded_param_typ_2', 'feature_asr__kontrola_trakcji_', 'feature_asystent_parkowania', 'feature_cd', 'feature_centralny_zamek', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_gniazdo_sd', 'feature_isofix', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_manualna', 'feature_ogrzewanie_postojowe', 'feature_podgrzewane_tylne_siedzenia', 'feature_relingi_dachowe', 'feature_tapicerka_skórzana', 'feature_tempomat', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'longitude', 'latitude', 'new_param_kolor_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_oferta_od_cat', 'new_param_opłata_początkowa_cat', 'new_param_wersja_cat', 'new_seller_name_cat', 'new_vin_flex_fuel_cat', 'new_vin_model_cat', 'new_vin_portrait_cat', 'new_vin_security_cat', 'ogloszenie_rok', 'param_bezwypadkowy', 'param_homologacja_ciężarowa', 'param_ile_lat_temu_produkcja', 'param_leasing', 'param_liczba_miejsc_2', 'param_liczba_pozostałych_rat_2', 'param_matowy', 'param_moc', 'param_perłowy', 'param_pierwsza_rejestracja_10_lat', 'param_przebieg', 'param_rok_produkcji', 'param_tuning', 'param_uszkodzony', 'param_vin_len'
# 'encoded_param_rodzaj_paliwa_2', 'feature_abs', 'feature_asr__kontrola_trakcji_', 'feature_bluetooth', 'feature_centralny_zamek', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_gniazdo_sd', 'feature_immobilizer', 'feature_isofix', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_manualna', 'feature_komputer_pokładowy', 'feature_ogrzewanie_postojowe', 'feature_podgrzewane_tylne_siedzenia', 'feature_poduszka_powietrzna_kierowcy', 'feature_radio_fabryczne', 'feature_szyberdach', 'feature_tapicerka_skórzana', 'feature_tempomat', 'feature_zmieniarka_cd', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'model_8790', 'new_param_kolor_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_model_pojazdu_cat', 'new_param_oferta_od_cat', 'new_param_opłata_początkowa_cat', 'new_param_wersja_cat', 'new_seller_name_cat', 'new_vin_flex_fuel_cat', 'new_vin_model_cat', 'new_vin_portrait_cat', 'new_vin_security_cat', 'ogloszenie_rok', 'param_bezwypadkowy', 'param_filtr_cząstek_stałych', 'param_homologacja_ciężarowa', 'param_ile_lat_temu_produkcja', 'param_liczba_pozostałych_rat_2', 'param_matowy', 'param_metalik', 'param_moc', 'param_perłowy', 'param_pierwsza_rejestracja_10_lat', 'param_pierwsza_rejestracja_ten_rok', 'param_przebieg', 'param_serwisowany_w_aso', 'param_tuning'
# 'encoded_param_rodzaj_paliwa_2', 'feature_abs', 'feature_bluetooth', 'feature_centralny_zamek', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_gniazdo_sd', 'feature_immobilizer', 'feature_isofix', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_manualna', 'feature_komputer_pokładowy', 'feature_ogrzewanie_postojowe', 'feature_podgrzewane_tylne_siedzenia', 'feature_poduszka_powietrzna_kierowcy', 'feature_radio_fabryczne', 'feature_szyberdach', 'feature_tapicerka_skórzana', 'feature_tempomat', 'feature_wielofunkcyjna_kierownica', 'feature_zmieniarka_cd', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'model_8790', 'new_param_kategoria_cat', 'new_param_kolor_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_model_pojazdu_cat', 'new_param_oferta_od_cat', 'new_param_opłata_początkowa_cat', 'new_seller_name_cat', 'new_vin_flex_fuel_cat', 'new_vin_model_cat', 'new_vin_portrait_cat', 'new_vin_security_cat', 'ogloszenie_rok', 'param_faktura_vat', 'param_filtr_cząstek_stałych', 'param_homologacja_ciężarowa', 'param_ile_lat_temu_produkcja', 'param_liczba_pozostałych_rat_2', 'param_matowy', 'param_metalik', 'param_moc', 'param_perłowy', 'param_pierwsza_rejestracja_10_lat', 'param_pierwsza_rejestracja_ten_rok', 'param_przebieg', 'param_serwisowany_w_aso', 'param_tuning'
#2539.081044523367
    'encoded_param_napęd_2', 'encoded_param_rodzaj_paliwa_2', 'feature_asystent_parkowania', 'feature_centralny_zamek', 'feature_czujnik_deszczu', 'feature_elektrochromatyczne_lusterko_wsteczne', 'feature_elektryczne_szyby_tylne', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_hud__wyświetlacz_przezierny_', 'feature_klimatyzacja_manualna', 'feature_mp3', 'feature_nawigacja_gps', 'feature_odtwarzacz_dvd', 'feature_ogranicznik_prędkości', 'feature_ogrzewanie_postojowe', 'feature_podgrzewana_przednia_szyba', 'feature_podgrzewane_tylne_siedzenia', 'feature_poduszka_powietrzna_kierowcy', 'feature_poduszki_boczne_przednie', 'feature_poduszki_boczne_tylne', 'feature_przyciemniane_szyby', 'feature_radio_fabryczne', 'feature_regulowane_zawieszenie', 'feature_tapicerka_skórzana', 'feature_wielofunkcyjna_kierownica', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'feature_światła_xenonowe', 'model_8790', 'new_param_emisja_co2_cat', 'new_param_kategoria_cat', 'new_param_rodzaj_paliwa_cat', 'new_param_skrzynia_biegów_cat', 'new_param_stan_cat', 'new_param_wersja_cat', 'new_seller_type_cat', 'new_seller_wojewodzctwo_cat', 'new_vin_portrait_cat', 'new_vin_security_cat', 'param_bezwypadkowy', 'param_faktura_vat', 'param_filtr_cząstek_stałych', 'param_liczba_miejsc_2', 'param_liczba_pozostałych_rat_2', 'param_miesięczna_rata_2', 'param_pierwsza_rejestracja_10_lat', 'param_pierwsza_rejestracja_date_na', 'param_przebieg', 'param_serwisowany_w_aso', 'param_vin_len'

])

# 'model_8790' = car_price_log_Vladimir_categories_xgb.csv; 5637.56722; "opis": "Vladimir z kategorialnymi mae na train_30"

black_list = ['car_id', 'price_details', 'price_value', 'price_value_log', 'price_value_pln', 'price_value_pln_log_transl', 'new_cat_price_value',
#               *[x for x in total.columns if x.startswith('model_')],
              'model_XGBRegressor_2195_24982_po_hiperopcie_caly_train', #train_100, przeuczony
              'model_XGBRegressor_3293_22097_', #train_100
              'longitude_na_mean', 'latitude_na_mean', #gorsze w xbfir
              *total.columns[total.isnull().any()],
             ]

feats3 = total.select_dtypes(include=[np.number, np.bool]).columns
feats3 = [x for x in feats3 if x not in black_list]
feats3 = np.sort(feats3)

print(total[start_vars].info(verbose=True))

print(total[feats3].info(verbose=True))
print(total[feats3].columns[total[feats3].isnull().any()])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160616 entries, 0 to 160615
Data columns (total 51 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   encoded_param_napęd_2                          160616 non-null  int64  
 1   encoded_param_rodzaj_paliwa_2                  160616 non-null  int64  
 2   feature_asystent_parkowania                    160616 non-null  bool   
 3   feature_centralny_zamek                        160616 non-null  bool   
 4   feature_czujnik_deszczu                        160616 non-null  bool   
 5   feature_elektrochromatyczne_lusterko_wsteczne  160616 non-null  bool   
 6   feature_elektryczne_szyby_tylne                160616 non-null  bool   
 7   feature_elektrycznie_ustawiane_fotele          160616 non-null  bool   
 8   feature_elektrycznie_ustawiane_lusterka        160616 non-null  bool   
 9   feature_hud__wyświetlacz_przezierny_ 

Index([], dtype='object')


In [ ]:
import contextlib
import warnings
warnings.filterwarnings("ignore")
import json

params = {    
    'n_jobs': 12, 

    'n_estimators': 90, 
    'learning_rate': 0.1, 
    'subsample': 0.5, 
    'colsample_bytree': 0.9,
    'max_depth': 10, 
}

mae_global_min = 2539.081044523367
DELTA = 500.0

big_iter = 0

current_vars=start_vars
all_vars = feats3 #maksymalny set

while True:

    pairs_so_far = []

    last_added = None
    last_removed = None

    last_trial = False
    big_iter += 1
    print("START iter=" + str(big_iter) + ', mae_global_min=' + str(mae_global_min))
    
    while True:
        with contextlib.redirect_stdout(None):
            current_vars, last_added, last_removed, pairs_so_far, last_trial =  mozo2.mutate_rand_feature_pairs(current_vars, all_vars, pairs_so_far)
#         print('pairs_so_far: ' + json.dumps(pairs_so_far, ensure_ascii=False, indent=4))
        
        print('added: ' + last_added)
        print('removed: ' + last_removed)
        print('len: ' + str(len(current_vars)))
        
        mae, r2, model, model_cl, learning_curve_plot = mozo2.train_and_submit(train_70, train_30, total.loc[total['price_value'].isna(),], 
                     current_vars, global_min, 
                     xgb.XGBRegressor, params, digitize=None, subfolder='XGBRegressor_parami_delta',
                     file_sufix = 'parami_delta_500', opis='start=2539.081044523367, delta 500', 
                     learning_curve = False, 
                     add_model_column_min = -1, total = total, 
                     save_min=mae_global_min, kaggle_min=-1)

        if mae < mae_global_min + DELTA:
            if (mae < mae_global_min):
                mae_global_min = mae
                print('Improvement: ' + last_removed + ' -> ' + last_added)
            else:
                print('Mutation: ' + last_removed + ' -> ' + last_added)
        else:
            current_vars = mozo2.replace_feature(current_vars, last_added, last_removed)
            print('Rollback: ' + last_added + ' -> ' + last_removed)
        print('=============================================' + str(mae_global_min)+ '=============================================')

        if last_trial == True: 
            print("END iter=" + str(big_iter) + ', mae_global_min=' + str(mae))
            break

START iter=1, mae_global_min=2539.081044523367
added: param_emisja_co2_2
removed: feature_ogranicznik_prędkości
len: 51
mea: 2639.249967192128
r2: 0.9794222344366376
Mutation: feature_ogranicznik_prędkości -> param_emisja_co2_2
=============================================2539.081044523367=============================================
added: feature_gniazdo_aux
removed: param_pierwsza_rejestracja_date_na
len: 51
mea: 2543.9834709433453
r2: 0.9796640833078992
Mutation: param_pierwsza_rejestracja_date_na -> feature_gniazdo_aux
=============================================2539.081044523367=============================================
added: price_currency_pln
removed: feature_elektrochromatyczne_lusterko_wsteczne
len: 51
mea: 2620.1913202318665
r2: 0.9790703301575059
Mutation: feature_elektrochromatyczne_lusterko_wsteczne -> price_currency_pln
=============================================2539.081044523367=============================================
added: feature_ogranicznik_prędkości
rem

mea: 2738.2212414501496
r2: 0.9824008455673314
Mutation: price_currency_pln -> model_XGBRegressor_3422_09438_start_vars_train_100
=============================================2539.081044523367=============================================
added: feature_światła_do_jazdy_dziennej
removed: feature_tempomat_aktywny
len: 51
mea: 2727.1373717310726
r2: 0.9825383696166993
Mutation: feature_tempomat_aktywny -> feature_światła_do_jazdy_dziennej
=============================================2539.081044523367=============================================
added: model_RandomForestRegressor_7089_27166_feats2
removed: feature_gniazdo_aux
len: 51
mea: 5618.1593555032905
r2: 0.9412641655841462
Rollback: model_RandomForestRegressor_7089_27166_feats2 -> feature_gniazdo_aux
=============================================2539.081044523367=============================================
added: feature_tapicerka_skórzana
removed: new_param_kolor_cat
len: 51
mea: 2752.048747058606
r2: 0.982320882017677
Mutation: ne

mea: 2661.2596273656322
r2: 0.9781081195555948
Mutation: feature_abs -> feature_podgrzewane_przednie_siedzenia
=============================================2539.081044523367=============================================
added: feature_czujniki_parkowania_tylne
removed: feature_radio_fabryczne
len: 51
mea: 2657.6821107597298
r2: 0.9781977027982423
Mutation: feature_radio_fabryczne -> feature_czujniki_parkowania_tylne
=============================================2539.081044523367=============================================
added: new_param_pierwsza_rejestracja_cat
removed: feature_światła_do_jazdy_dziennej
len: 51
mea: 2717.1736947765658
r2: 0.9780805938174496
Mutation: feature_światła_do_jazdy_dziennej -> new_param_pierwsza_rejestracja_cat
=============================================2539.081044523367=============================================
added: param_możliwość_finansowania
removed: param_akryl__niemetalizowany_
len: 51
mea: 2743.9540086706184
r2: 0.9777806728737557
Mutation: par

mea: 2675.6101818595866
r2: 0.9782341241899115
Mutation: param_rok_produkcji -> latitude
=============================================2539.081044523367=============================================
added: feature_radio_fabryczne
removed: feature_komputer_pokładowy
len: 51
mea: 2667.746388344189
r2: 0.9782741428703488
Mutation: feature_komputer_pokładowy -> feature_radio_fabryczne
=============================================2539.081044523367=============================================
added: feature_bluetooth
removed: latitude
len: 51
mea: 2670.8042960018743
r2: 0.9780770860212497
Mutation: latitude -> feature_bluetooth
=============================================2539.081044523367=============================================
added: new_vin_where_cat
removed: param_emisja_co2_2
len: 51
mea: 2675.5654359163755
r2: 0.9778760154708253
Mutation: param_emisja_co2_2 -> new_vin_where_cat
=============================================2539.081044523367============================================

mea: 2653.16077481335
r2: 0.9790580046876413
Mutation: param_zarejestrowany_w_polsce -> feature_tempomat
=============================================2539.081044523367=============================================
added: feature_immobilizer
removed: param_emisja_co2_2
len: 51
mea: 2551.857159647688
r2: 0.9795782583679662
Mutation: param_emisja_co2_2 -> feature_immobilizer
=============================================2539.081044523367=============================================
added: param_perłowy
removed: feature_bluetooth
len: 51
mea: 2643.4342324327718
r2: 0.9784826532912342
Mutation: feature_bluetooth -> param_perłowy
=============================================2539.081044523367=============================================
added: param_faktura_vat
removed: feature_czujnik_zmierzchu
len: 51
mea: 2588.9591559463543
r2: 0.9792058048168589
Mutation: feature_czujnik_zmierzchu -> param_faktura_vat
=============================================2539.081044523367============================

...

mea: 5774.497636085187
r2: 0.9408019238782153
Rollback: model_XGBRegressor_5856_36229_ -> feature_asystent_pasa_ruchu
=============================================2520.234407465377=============================================
added: feature_elektrycznie_ustawiane_lusterka
removed: created_at_unix_na_mean
len: 51
mea: 2878.1813867539013
r2: 0.9782481005499941
Mutation: created_at_unix_na_mean -> feature_elektrycznie_ustawiane_lusterka
=============================================2520.234407465377=============================================
added: feature_elektrochromatyczne_lusterko_wsteczne
removed: feature_radio_niefabryczne
len: 51
mea: 2884.2085580362664
r2: 0.9774344406196458
Mutation: feature_radio_niefabryczne -> feature_elektrochromatyczne_lusterko_wsteczne
=============================================2520.234407465377=============================================
added: param_zarejestrowany_w_polsce
removed: feature_poduszka_powietrzna_chroniąca_kolana
len: 51
mea: 2720.4177184

mea: 2745.9813248519613
r2: 0.9786435026238927
Mutation: feature_asystent_parkowania -> new_vin_security_cat
=============================================2520.234407465377=============================================
added: feature_cd
removed: ogloszenie_dzien_tyg
len: 51
mea: 2776.4115359233724
r2: 0.9785243557379956
Mutation: ogloszenie_dzien_tyg -> feature_cd
=============================================2520.234407465377=============================================
added: param_rok_produkcji_10_lat
removed: feature_immobilizer
len: 51
mea: 2747.1058489444817
r2: 0.978695621998519
Mutation: feature_immobilizer -> param_rok_produkcji_10_lat
=============================================2520.234407465377=============================================
added: feature_kamera_cofania
removed: feature_podgrzewane_tylne_siedzenia
len: 51
mea: 2759.8215079860092
r2: 0.9784217783338219
Mutation: feature_podgrzewane_tylne_siedzenia -> feature_kamera_cofania
========================================

mea: 2746.9063517615323
r2: 0.9787043250080284
Mutation: feature_tempomat_aktywny -> param_rok_produkcji_ten_rok
=============================================2520.234407465377=============================================
added: new_param_stan_cat
removed: param_przebieg
len: 51
mea: 2746.8427739021836
r2: 0.9781669317152744
Mutation: param_przebieg -> new_param_stan_cat
=============================================2520.234407465377=============================================
added: param_vat_discount
removed: feature_szyberdach
len: 51
mea: 2772.9452420901616
r2: 0.9779611293834521
Mutation: feature_szyberdach -> param_vat_discount
=============================================2520.234407465377=============================================
added: feature_światła_przeciwmgielne
removed: feature_światła_led
len: 51
mea: 2781.7781569711196
r2: 0.9778767880341737
Mutation: feature_światła_led -> feature_światła_przeciwmgielne
=============================================2520.234407465377===

mea: 2732.2827765759703
r2: 0.9808128558094383
Mutation: new_param_wartość_wykupu_cat -> feature_klimatyzacja_czterostrefowa
=============================================2520.234407465377=============================================
added: feature_asystent_parkowania
removed: param_vat_discount
len: 51
mea: 2712.2946906103493
r2: 0.9802474520340672
Mutation: param_vat_discount -> feature_asystent_parkowania
=============================================2520.234407465377=============================================
added: new_param_skrzynia_biegów_cat
removed: new_param_kod_silnika_cat
len: 51
mea: 2733.531232150661
r2: 0.9801761552890618
Mutation: new_param_kod_silnika_cat -> new_param_skrzynia_biegów_cat
=============================================2520.234407465377=============================================
added: feature_immobilizer
removed: feature_dach_panoramiczny
len: 51
mea: 2728.74284934568
r2: 0.9802223398889626
Mutation: feature_dach_panoramiczny -> feature_immobilizer
====

mea: 2732.5522071383853
r2: 0.9807392427567526
Mutation: new_seller_wojewodzctwo_cat -> feature_kamera_cofania
=============================================2520.234407465377=============================================
added: feature_kurtyny_powietrzne
removed: param_wartość_wykupu_2
len: 51
mea: 2735.659917566452
r2: 0.9799164786914459
Mutation: param_wartość_wykupu_2 -> feature_kurtyny_powietrzne
=============================================2520.234407465377=============================================
added: feature_asystent_pasa_ruchu
removed: feature_tapicerka_skórzana
len: 51
mea: 2734.2439178131526
r2: 0.9798796274096765
Mutation: feature_tapicerka_skórzana -> feature_asystent_pasa_ruchu
=============================================2520.234407465377=============================================
added: feature_łopatki_zmiany_biegów
removed: feature_światła_przeciwmgielne
len: 51
mea: 2734.379750110772
r2: 0.9797753757117653
Mutation: feature_światła_przeciwmgielne -> feature_łopat

In [70]:
from kaggle.api.kaggle_api_extended import KaggleApi
kaggle_api = KaggleApi()
kaggle_api.authenticate()

leaderboard = kaggle_api.competition_view_leaderboard('dw-car-price-prediction')
print(pd.DataFrame(leaderboard['submissions'])[['teamName', 'submissionDate', 'score']].head())

          teamName                submissionDate       score
0       adamskorek  2020-10-31T23:26:16.9833333Z  5156.58793
1           pelele  2020-10-31T19:46:31.2633333Z  5369.00636
2             mozo  2020-10-30T16:38:46.7333333Z  5466.72056
3  Adrian Mich***i       2020-10-31T12:30:08.39Z  5578.96222
4   Remek Półtorak  2020-10-31T19:09:20.1866667Z  5581.85197


In [38]:
# model = RandomForestRegressor(n_estimators=50, max_depth=7).fit(X_sub_train, y_sub_train) 
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.svm import SVC


# perm = PermutationImportance(model).fit(X_sub_test, y_sub_test) 
eli5.show_weights(perm, feature_names=random_feats, top=50) 


/home/conda/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/conda/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


NameError: name 'perm' is not defined

save_model(file_sufix='test', subfolder='', test = total.loc[total['price_value'].isna()],
               feats=current_vars, global_min=global_min, model=model, shift = 0, digitize=False, mae=mae, r2=r2,
               details='test zapisu', learning_curve_plot=None, hist_plt=None, kaggle=None)
